In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [86]:
block_size = 3
X, Y = [], []
for w in words:
    context = [0]* block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [87]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [16]:
C = torch.randn((27, 2))

In [24]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [25]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [47]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [48]:
h

tensor([[-0.7351, -0.2610,  0.9476,  ...,  0.7739, -0.7264,  0.5427],
        [-0.9544, -0.3014,  0.9673,  ...,  0.9629, -0.2121,  0.8956],
        [-0.8578,  0.9773, -0.7375,  ..., -0.7610,  0.9054,  0.7567],
        ...,
        [-0.6132,  0.9463,  0.9382,  ...,  0.8698,  0.8111,  0.9381],
        [-0.9593,  0.9744,  0.4783,  ...,  0.6645, -0.4978, -0.1294],
        [-0.1057, -0.9832, -0.6272,  ...,  0.8646, -0.7699,  0.5642]])

In [49]:
h.shape

torch.Size([32, 100])

In [50]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [51]:
logits = h @ W2 + b2

In [52]:
logits.shape

torch.Size([32, 27])

In [53]:
coutns = logits.exp()

In [54]:
prob = coutns / coutns.sum(1, keepdim=True)


In [55]:
prob.shape

torch.Size([32, 27])

In [60]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(16.3871)

In [61]:
# ------ now made respectable :) --------

In [88]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [89]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1,W2, b2]

In [90]:
sum(p.nelement() for p in parameters)

3481

In [91]:
for p in parameters:
    p.requires_grad = True

for _ in range(10):
    # forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h@W2+b2
    # loss
    loss = F.cross_entropy(logits,Y)
    print(loss.item())
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.1 * p.grad


In [95]:
torch.randint(0, X.shape, (32,))

tensor([0, 2, 3, 0, 2, 2, 3, 4, 0, 0, 4, 4, 1, 0, 1, 4, 0, 0, 2, 3, 3, 2, 3, 3,
        0, 3, 4, 1, 2, 0, 4, 4])

In [ ]:
# 43:03